<a href="https://colab.research.google.com/github/descobarsalce/Classifier_BERT/blob/main/RunClassifier_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# prompt: transfer all data from local folder on my physical drive located in "C:\Users\descobar-salce\OneDrive - Eastern Research Group\Documents\TextClassification_Python" to my google colab folder. This is not pandas, but several .py files in the folder

!cp -r C:\Users\descobar-salce\OneDrive\Documents\TextClassification_Python/* /content/


cp: cannot stat 'C:Usersdescobar-salceOneDriveDocumentsTextClassification_Python/*': No such file or directory


In [ ]:
!pip install -r requirements.txt

In [ ]:
%load_ext autoreload
%autoreload 2

In [6]:
%cd /home
!mkdir TextClassification
%cd /home/TextClassification

/home
mkdir: cannot create directory ‘TextClassification’: File exists
/home/TextClassification


In [8]:
import pandas as pd
from ClassifierBERT import ClassifierBERT
from html_parser import HTML_folder_parser
import pickle, json

ModuleNotFoundError: ignored

In [ ]:
project_prefix = "WHD-2022-0003"
folder_path = "DATA/WHD-2022-0003_comments_TEMP"  # Replace with the path comments' folder

# Load processed labels from
labeled_data_path = 'DATA/labeled_data_' + project_prefix
labeled_data = pd.read_pickle(labeled_data_path)


In [ ]:

# Load data

MyParser = HTML_folder_parser(folder_path, project_prefix)

unique_files = MyParser.find_files_and_attachments()

# CHECK: remove the sample from this
#unique_files = unique_files.sample(5000)

unique_entries = MyParser.read_and_parse_html(unique_files)

# Filter those values that this is not processing yet:
exclude_values = ['jpg', 'png', 'docx'] # CHECK!
unique_entries[~unique_entries['extension'].isin(exclude_values)]

pd.to_pickle(unique_entries, 'DATA/processed_html_data')

In [ ]:
# Identify rows with removed data. Just for tracking (this info is not used).
print(unique_entries['comment'].str.contains("[EMAIL REMOVED]", regex=False).value_counts())
print(unique_entries['comment'].str.contains("[NAME REMOVED]", regex=False).value_counts())
print(unique_entries['comment'].str.contains("[ADDRESS REMOVED]", regex=False).value_counts())


#CHECK! See attached file(s) ARE NOT REPEATED - UNLESS PROVEN OTHERWISE IN THEIR text
#HOW DO I HANDLE TEXT INSIDE PDFS FOR LABELING?

# Create a Series with the same index as unique_entries, containing the count of occurrences for each entry_code
entry_code_counts = unique_entries['entry_code'].map(unique_entries['entry_code'].value_counts(dropna=False))

# Now use this Series for boolean indexing
duplicate_entry_codes = unique_entries[entry_code_counts > 1]

# Now, duplicate_entry_codes should contain the rows from unique_entries where entry_code occurs more than once


In [ ]:

unique_entries = pd.read_pickle('DATA/processed_html_data')

# Run deduplication comparing to find elements sharing common text. This approach uses inverse term frequency to give more weight to elements that appear less often. Then it generates vector representations of the text and compare their cosine similarity.

from DeDuplication import RecordLinkage

# Create an instance of the RecordLinkage class
DeDuplicator = RecordLinkage(unique_entries, 'comment', n_gramsize=[8])

# Perform de-duplication
deduplicated = DeDuplicator.deduplication(ntop=100000, lower_bound=0.6, max_matches=None, n_grams=[8], full_words=True)
# ntop parameters is in case i want a maximum number of entries for each to be matched, but in principle there is no limit to the number of matches here

# Now we get the mapping of the new categories to group_ids
new_group_ids = DeDuplicator.generate_group_ids(deduplicated)
unique_entries = pd.merge(unique_entries, new_group_ids,
                         how='outer',
                         on='comment')


In [ ]:

# Descriptive statistics on repeated text or mass campaigns
unique_entries['repetitions_count'] = unique_entries.groupby('group_id')['group_id'].transform('count')
unique_entries['is_campaign'] = unique_entries['repetitions_count']>5
print(unique_entries['is_campaign'].value_counts())
unique_entries['repetitions_count'].value_counts().hist()

In [ ]:

# Now we merge the unlabeled data in all entries with those already labeled:

merged_labeled_data = pd.merge(left=unique_entries, right=labeled_data,
                       left_on=['entry_code'],
                       right_on=['entry_code_short'],
                       indicator='label_merge',
                       how='outer')

merged_labeled_data = merged_labeled_data[merged_labeled_data['parsed_html'].isna()==False]
# Keep unlabeled examples to test model below:
unlabeled_data = merged_labeled_data[merged_labeled_data['label_merge']=='left_only']
# Separate labeled data to train and test model:
merged_labeled_data = merged_labeled_data[merged_labeled_data['label_merge']=='both']


In [ ]:

pd.to_pickle(unlabeled_data, 'DATA/unlabeled_data_pickle')

model_data = merged_labeled_data[['entry_code_short', 'DESCRIPT', 'numeric_labels', 'comment']]
model_data = merged_labeled_data[['comment', 'numeric_labels']]
pd.to_pickle(model_data, 'DATA/model_data_pickle')

# Starting from processed data:


In [ ]:

# To load the dictionary back from the JSON file:
with open('DATA/description_mapping.json', 'r') as file:
    description_mapping = json.load(file)

unlabeled_data = pd.read_pickle('DATA/unlabeled_data_pickle')

model_data = pd.read_pickle('DATA/model_data_pickle')
text_varname = 'comment'
labels_varname = 'numeric_labels'

In [ ]:

# Instantiate the BERT Classifier
MyClassifier = ClassifierBERT(model_data, text_varname, labels_varname, model='distilbert-base-uncased')


new_data = MyClassifier.generate_augmented_data(train_df, len(train_df), shuffle=True)

# Train only non-BERT layers first:
MyClassifier.freeze_BERT_layers(unfreeze=False)
MyClassifier.train_model(epochs=30, batch_size=16, train_size=0.7, test_size=0.15, val_size=0.15,
                         initial_learning_rate=3e-5)

# Train the model with all layers:
MyClassifier.freeze_BERT_layers(unfreeze=True)
MyClassifier.train_model(epochs=1, batch_size=16, train_size=0.7, test_size=0.15, val_size=0.15,
                         initial_learning_rate=3e-5)

In [ ]:

# Save trained model:
MyClassifier.save_model('DATA/Trained_Classifier_SHORT')

In [ ]:

# Load model:
MyClassifier_loaded = ClassifierBERT.load_model('DATA/Trained_Classifier_SHORT')


In [ ]:
# Make predictions on unlabeled data
unlabeled_data_unique = unlabeled_data[['comment','group_id']].drop_duplicates(subset='group_id', keep='first', inplace=False)
unlabeled_text_varname = 'comment'
unlabeled_data_unique = unlabeled_data_unique.sample(500)

df_human_readable_labels = MyClassifier_loaded.predict_unlabeled_data(unlabeled_data_unique, unlabeled_text_varname, description_mapping)

